In [ ]:
import pandas as pd
import numpy as np

# Load the data from the CSV file
df = pd.read_csv('enriched_merged_useragent_immoscout24.csv', sep=';')

def print_dataframe(df):
    # Find the maximum length of data in each column
    max_lengths = [max([len(str(s)) for s in df[col].values] + [len(col)]) for col in df.columns]
    
    # Create a horizontal line based on the lengths
    h_line = '-' * (sum(max_lengths) + 3 * len(df.columns) - 1)
    
    # Print the header
    header = " | ".join([col.ljust(max_lengths[i]) for i, col in enumerate(df.columns)])
    print(h_line)
    print(header)
    print(h_line)
    
    # Print each row of data
    for index, row in df.iterrows():
        # Each row entry is padded to align with the column width
        row_str = " | ".join([str(row[col]).ljust(max_lengths[i]) for i, col in enumerate(df.columns)])
        print(row_str)
    print(h_line)

# Display the DataFrame
print(df.dtypes)
print_dataframe(df)




In [ ]:
# Entfernen von "CHF", Kommas und dem Punkt am Ende sowie anderen nicht-numerischen Zeichen
df['Price'] = df['Price'].replace({'CHF': '', ',': '', '.–': '', '_': ''}, regex=True)
df['Rooms'] = df['Rooms'].replace({'rooms': ''}, regex=True)
df['Living Space (sqm)'] = df['Living Space (sqm)'].replace({'m²': ''}, regex=True)

# Convert the 'price' column to a numeric type
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
df['Rooms'] = pd.to_numeric(df['Rooms'], errors='coerce')
df['Living Space (sqm)'] = pd.to_numeric(df['Living Space (sqm)'], errors='coerce')


print(df.dtypes)


In [ ]:
#Dublikate entfernen
df = df.drop_duplicates()
duplicate_count=df.duplicated().sum()
print(df)
print(f"Anzahl der verbleibenden Duplikate: {duplicate_count}")



In [ ]:
#Fehlende Werte finden / Nan entfernen
# Prüfen, welche Datensätze in der Spalte 'Preis' NaN-Werte enthalten
nan_rooms = df['Rooms'].isna()
print(df[nan_rooms])

#Wie viele Nan werte gibt es insgesamt?
print(df.isna().sum())

#Alle Nan Werte aus den wichtigsten Prädikatoren entfernen
df_clean = df.dropna(subset=['Price', 'Rooms', 'Living Space (sqm)'])

# Überprüfen der neuen Datengröße und ob noch NaN-Werte in den genannten Spalten vorhanden sind.
print(df.shape)
print(df_clean.shape)
print(df_clean.isna().sum())




In [ ]:
df_clean['Price_per_SquareMeter'] = df_clean['Price'] / df_clean['Living Space (sqm)']

# Überprüfen Sie die ersten paar Zeilen des DataFrames, um die neue Spalte zu sehen
print_dataframe(df_clean)
print(df_clean.dtypes)


In [7]:
# Anwendung von isinf auf spezifische Spalten
inf_mask = np.isinf(df_clean['Distance from nearest station (m)'])
print(inf_mask.sum())
df_clean = df_clean[~inf_mask]
print(df_clean.shape)

0
(13140, 9)


In [10]:
df_clean.to_csv('LAST_Final_cleaned_immodata.csv', sep=';', index=False)


In [10]:

# Berechnen des IQR
Q1 = df['Price'].quantile(0.25)
Q3 = df['Price'].quantile(0.75)
IQR = Q3 - Q1

# Definition der Grenzen für Ausreißer
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

#Ausreisser entfernen
df_filtered = df_clean[(df_clean['Price'] >= lower_bound) & (df_clean['Price'] <= upper_bound)]

# Berechnen des IQR
Q1 = df['Rooms'].quantile(0.25)
Q3 = df['Rooms'].quantile(0.75)
IQR = Q3 - Q1

# Definition der Grenzen für Ausreißer
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

#Ausreisser entfernen
df_double_filtered = df_filtered[(df_clean['Rooms'] >= lower_bound) & (df_clean['Rooms'] <= upper_bound)]
df_double_filtered.to_csv('Double_Final_Filtered_immodata.csv', sep=';', index=False)




C:\Users\schae\AppData\Local\Temp\ipykernel_10360\1442309437.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_double_filtered = df_filtered[(df_clean['Rooms'] >= lower_bound) & (df_clean['Rooms'] <= upper_bound)]
